In [20]:
#!pip install mesa seaborn --quiet

In [21]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import SimultaneousActivation
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

import random

In [ ]:
class LuigiAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.points = 4
        self.saved_points = 0  # Puntos ahorrados de turnos anteriores
        self.carrying_portrait = None  # Para saber si está cargando un retrato verdadero

    def get_adjacent_agents(self):
        adjacent_agents = []
        possible_moves = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
        for pos in possible_moves:
            adjacent_agents.extend(self.model.grid.get_cell_list_contents([pos]))
        return adjacent_agents

    def decide_to_save_points(self):
        # Ahorra si no hay tareas críticas cerca y no tiene un retrato verdadero
        if self.carrying_portrait:
            return False  # No ahorra si está cargando un retrato verdadero
        adjacent_agents = self.get_adjacent_agents()
        ghosts_nearby = any(ghost for ghost in self.model.ghosts if ghost["pos"] in [agent.pos for agent in adjacent_agents])
        portraits_nearby = any(portrait for portrait in self.model.portraits if portrait["pos"] in [agent.pos for agent in adjacent_agents])
        return not ghosts_nearby and not portraits_nearby  # Ahorra si no hay fantasmas ni retratos cerca

    def move(self):
        if self.points >= 1:
            possible_moves = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
            new_position = self.random.choice(possible_moves)
            self.model.grid.move_agent(self, new_position)
            self.points -= 1  # Costo de moverse

    def move_with_portrait(self):
        if self.points >= 2:
            possible_moves = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
            new_position = self.random.choice(possible_moves)
            self.model.grid.move_agent(self, new_position)
            self.points -= 2  # Costo de moverse con un retrato

    def toggle_door(self, door):
        if self.points >= 1:
            door["is_open"] = not door["is_open"]
            self.points -= 1
            state = "abierta" if door["is_open"] else "cerrada"
            print(f"Luigi {self.unique_id} ha {state} la puerta en {self.pos}.")

    def perform_knocked(self, ghost):
        if self.points >= 1:
            ghost["points"] -= 1
            self.points -= 1
            print(f"Luigi {self.unique_id} ha noqueado un fantasma en {self.pos}")
            self.check_ghost_health(ghost)

    def perform_exorcism(self, ghost):
        if self.points >= 2 and ghost["points"] == 2:
            ghost["points"] -= 2
            ghost["is_dead"] = True
            self.points -= 2
            print(f"Luigi {self.unique_id} ha absorbido un fantasma en {self.pos}")

    def check_ghost_health(self, ghost):
        if ghost["points"] <= 0:
            ghost["is_dead"] = True
            print(f"El fantasma en {ghost['pos']} ha sido eliminado.")

    def examine_portrait(self, portrait):
        if self.points >= 1:
            portrait["is_revealed"] = True
            self.points -= 1
            if portrait["is_fake"]:
                print(f"El retrato en {self.pos} ha sido revelado como un espejo (falsa alarma).")
            else:
                print(f"El retrato en {self.pos} ha sido revelado como un personaje real.")
                self.carrying_portrait = portrait  # Luigi ahora lleva un retrato verdadero

    def prioritize_saving_portrait(self):
        """
        Prioriza moverse hacia la puerta más cercana para salvar el retrato.
        """
        if not self.carrying_portrait:
            return False  # No hace nada si no está cargando un retrato verdadero

        # Buscar la puerta más cercana
        doors = [door for door in self.model.doors if door["is_open"]]
        if not doors:
            print("No hay puertas abiertas disponibles para salvar el retrato.")
            return False

        closest_door = min(doors, key=lambda door: self.model.grid.get_distance(self.pos, door["pos"]))

        # Moverse hacia la puerta respetando el costo de movimiento con retrato
        while self.points >= 2 and self.pos != closest_door["pos"]:
            next_step = self.get_next_step_towards(self.pos, closest_door["pos"])
            if next_step:
                self.model.grid.move_agent(self, next_step)
                self.points -= 2  # Costo de moverse con retrato

        # Si llegó a la puerta
        if self.pos == closest_door["pos"]:
            print(f"Luigi {self.unique_id} ha salvado el retrato en la puerta {self.pos}.")
            self.carrying_portrait = None  # Retrato salvado
            return True
        return False

    def get_next_step_towards(self, start, goal):
        """
        Calcula el siguiente paso hacia el objetivo.
        """
        path = self.model.grid.get_path(start, goal)
        return path[1] if len(path) > 1 else None  # Devuelve el siguiente paso en la ruta

    def step(self):
        # Sumar puntos ahorrados al inicio del turno
        self.points += self.saved_points
        self.saved_points = 0  # Resetear puntos ahorrados

        # Si lleva un retrato verdadero, prioriza salvarlo
        if self.carrying_portrait:
            if self.prioritize_saving_portrait():
                return  # Si logra salvar el retrato, termina el turno

        # Decidir si ahorrar o gastar puntos
        if self.decide_to_save_points():
            self.saved_points += self.points
            print(f"Luigi {self.unique_id} decide ahorrar {self.points} puntos.")
            self.points = 0  # Todos los puntos se transfieren a ahorros
        else:
            if self.points >= 1:
                self.move()  # Si no decide ahorrar, intenta moverse
            self.interact()  # Luego interactúa



In [23]:
def get_grid(model):
    # Inicializamos una cuadrícula vacía
    grid = np.empty((model.grid.width, model.grid.height), dtype=object)
    
    # Iteramos sobre todas las celdas de la cuadrícula
    for x in range(model.grid.width):
        for y in range(model.grid.height):
            # Obtenemos los agentes en la celda actual
            agents_in_cell = model.grid.get_cell_list_contents([(x, y)])
            
            # Asignamos una lista de agentes en esa celda
            grid[x, y] = agents_in_cell
            
    return grid

In [ ]:
class MansionModel(Model):
    def __init__(self, width, height, luigi, ghost, walls, doors, doors_connected, entrances, portrait):
        super().__init__()
        self.grid = SingleGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)
        self.ghosts = [] # Manejar fantasmas dentro del modelo
        self.portraits = [] # Manejar retratos dentro del modelo
        self.datacollector = DataCollector(
            model_reporters={
                "Grid": get_grid,
                "Steps": lambda model: model.steps,
            },
        )
        self.num_agents = luigi  # Número de agentes fantasmas y retratos
        self.create_agents()

        self.steps = 0

    def create_agents(self):
        # Crear fantasmas y retratos directamente dentro del modelo
        for i in range(self.num_agents):
            # Fantasmas
            ghost_data = {
                "points": 1, # Los fantasmas empiezan como Green Ghosts
                "is_dead": False, # Los fantasmas empiezan vivos
                "pos": (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
            }
            self.ghosts.append(ghost_data)

            # Retratos
            portrait_data = {
                "is_revealed": False, # Todos empiezan sin descubrir
                "is_fake": self.random.choice([True, False]),  # Ciertos son reales, otros no
                "damaged": False, # Todos empiezan sin daños
                "pos": (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
            }
            self.portraits.append(portrait_data)

        # Crear Luigi
        luigi_pos = (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
        luigi = LuigiAgent(self.next_id(), self)
        self.grid.place_agent(luigi, luigi_pos)
        self.schedule.add(luigi)

    def get_grid(self, model):
        # Devuelve una representación de la cuadrícula
        grid = np.zeros((model.grid.width, model.grid.height), dtype=object)
        
        for cell in range(model.grid.width):
            for row in range(model.grid.height):
                cell_content = model.grid.get_cell_list_contents([(cell, row)])
                grid[cell][row] = cell_content  # Guarda los agentes de la celda
        return grid

    def step(self):
    # Realiza un paso en la simulación, actualizando el estado de todos los agentes
        for cell in self.grid.coord_iter():
            cell_agents = self.grid.get_cell_list_contents([cell[1]])  # Obtener agentes en todas las celdas
            for agent in cell_agents:
                if isinstance(agent, LuigiAgent):
                    agent.step() # Los agentes Luigi realizan su acción
        
        # Manejar lógica de los fantasmas para cada turno
        for ghost in self.ghosts:
            self.handle_ghost(ghost)

        # Manejar lógica para los retratos para cada turno
        for portrait in self.portraits:
            self.handle_portrait(portrait)

        self.steps += 1 # Se terminó un turno
        self.datacollector.collect(self)

    def handle_ghost(self, ghost):
        # Lógica para manejar los fantasmas dentro del modelo
        if not ghost["is_dead"]:
            # Si el fanstasma termina el turno con un punto, subir de nivel
            # Empezar a partir del segundo turno del juego
            if ghost["points"] == 1 and self.steps > 1:
                ghost["points"] = 2
                print(f"El fantasma en {ghost['pos']} se ha convertido en fantasma rojo.")
            # Si el fantasma está muerto, no hacer nada
            elif ghost["points"] <= 0:
                ghost["is_dead"] = True
                print(f"El fantasma en {ghost['pos']} ha sido eliminado.")

    def handle_portrait(self, portrait):
        # Lógica para manejar los retratos dentro del modelo
        if portrait["is_revealed"]:
            if portrait["is_fake"]:
                print(f"El retrato en {portrait['pos']} ha sido revelado como falso.")
            else:
                print(f"El retrato en {portrait['pos']} ha sido revelado correctamente.")
        if portrait["damaged"]:
            print(f"El retrato en {portrait['pos']} ha sido dañado.")


In [ ]:
GRID_SIZE = 10
AGENTS = 6
MAX_ENERGY = 120

# Inicializar model
model = MansionModel(GRID_SIZE, GRID_SIZE, AGENTS, MAX_ENERGY)

# Ejecutar modelo hasta que se pare solo
model.step()

# Mostrar cuántos turnos tomó
print("Total turns:", model.steps)

Total turns: 1


In [26]:
all_grid = model.datacollector.get_model_vars_dataframe()
all_grid.head(11)

,Grid,Steps
0,"[[[], [], [], [], [], [], [], [], [], []], [[]...",1


In [ ]:
# MODELO
class Mansion(Model):
    def __init__(self, width, height, luigi, ghost, walls, doors, doors_connected, entrances, portrait):
        super().__init__()

        # Parámetros de entrada almacenados
        self.doors = doors
        self.doors_connected = doors_connected
        self.entrances =  entrances
        self.portrait = portrait

        # Atributos para calcular las probabilidades de colocar un nuevo POI
        self.residual_victims = 12
        self.residual_alarms = 6

        # Atributos condicionales para ganar/perder
        self.total_deaths = 0
        self.total_saved_victims = 0
        self.damage = 0

        # Atributos para intercalar y ejecutar steps entre los agentes y el modelo
        self.index = 0
        self.schedule = SimultaneousActivation(self)

        # Atributo que nos permite saber los últimos movimientos
        self.moves = []

        # Atributo que inicializa la posición del agente como nula dentro del modelo
        self.agent_moves = None

        # Atributo que nos permite saber la posición del agente
        self.agent_pos = []

        # Atributos que funcionan como las matrices para graficar y verificar si hay fuegos, humos y POIs
        # Rejilla
        self.grid = MultiGrid(height, width, False)

        # Paredes
        self.walls_grid = np.zeros((height, width), dtype=object)
        for y in range(6):
            for x in range(8):
                self.walls_grid[y, x] = walls[y][x]

        # Fuegos y humos
        self.ghosts_grid = np.zeros((height, width))
        for (x, y) in ghost:

            # Validar si las coordenadas están dentro del rango
            if x < height and y < width:
                self.ghosts_grid[x][y] = 2

        # POI
        self.portrait_grid = np.zeros((height, width))
        for i in range(3):
            y_portrait = self.portrait[i][0]
            x_portrait = self.portrait[i][1]
            portrait_type = self.portrait[i][2]
            self.portrait_grid[y_portrait, x_portrait] = 3
            if portrait_type:
                self.residual_victims -= 1
            else:
                self.residual_alarms -= 1

        # Colocar los FireFighterAgent
        for i in range(luigi):
            agent = LuigiAgent(i, self)
            pos = self.random.choice(self.entrances)
            self.agent_pos.append([agent.unique_id, pos[0], pos[1]])
            self.grid.place_agent(agent, pos)
            self.schedule.add(agent)
            self.assign_initial_statement()

        # Atributo que nos permite recabar información y mandar a llamar a la función para graficar la simulación
        self.datacollector = DataCollector(model_reporters={"Grid": get_grid})


    def step(self):
        self.moves = []
        if self.index > 5:
            self.index = 0
        specific_agent = self.schedule.agents[self.index]
        self.datacollector.collect(self)
        specific_agent.step()
        self.index += 1
        self.agent_moves = specific_agent.moves


    def calculate_distance(self, pos1, pos2):

        # Calcula la distancia Euclidiana entre dos posiciones.
        return math.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)


    def assign_initial_statement(self):

        # Crear un diccionario para almacenar los agentes más cercanos a cada POI
        portrait_assignments = {portrait: [] for portrait in self.portrait}

        # Crear una lista para almacenar los agentes y sus distancias a los POIs
        agent_distances = []

        # Para cada agente en el modelo
        for agent in self.schedule.agents:

            # Calcular la distancia a cada POI
            distances = [self.calculate_distance(agent.pos, portrait) for portrait in self.portrait]

            # Encontrar el POI más cercano
            min_distance = min(distances)
            closest_portrait = self.portrait[distances.index(min_distance)]

            # Almacenar el agente, la distancia al POI más cercano y el POI
            agent_distances.append((agent, min_distance, closest_portrait))

        # Ordenar los agentes por la distancia al POI más cercano
        agent_distances.sort(key=lambda x: x[1])

        # Seleccionar solo la mitad de los agentes más cercanos a un POI
        num_agents_to_assign = 4
        agents_to_assign = agent_distances[:num_agents_to_assign]

        # Asignar los agentes seleccionados a los POIs
        for agent, _, closest_portrait in agents_to_assign:
            portrait_assignments[closest_portrait].append(agent)

        # Actualizar el estado de los agentes asignados a los POIs
        for portrait, agents in portrait_assignments.items():
            for agent in agents:

                # Si el agente está asignado a un POI, debe buscar el POI
                agent.state = 'heading_to_portrait'

        # Los agentes que no están asignados a un POI deben apagar el fuego
        for agent in self.schedule.agents:
            if agent.state != 'heading_to_portrait':
                agent.state = 'capturing'

    def assign_initial_statement_except_entrance(self):

            # Crear un diccionario para almacenar los agentes más cercanos a cada POI
            portrait_assignments = {portrait: [] for portrait in self.portrait}

            # Crear una lista para almacenar los agentes y sus distancias a los POIs
            agent_distances = []

            for agent in self.schedule.agents:

                # Excluir a los agentes que están en el estado 'heading to entrance'
                if agent.state != 'heading to entrance':

                    # Borrar el path actual del agente
                    agent.path = []

            # Para cada agente en el modelo
            for agent in self.schedule.agents:

                # Excluir a los agentes que están en el estado 'heading to entrance'
                if agent.state != 'heading to entrance':

                    # Calcular la distancia a cada POI
                    distances = [self.calculate_distance(agent.pos, portrait) for portrait in self.portrait]

                    # Encontrar el POI más cercano
                    min_distance = min(distances)
                    closest_portrait = self.portrait[distances.index(min_distance)]

                    # Almacenar el agente, la distancia al POI más cercano y el POI
                    agent_distances.append((agent, min_distance, closest_portrait))

            # Ordenar los agentes por la distancia al POI más cercano
            agent_distances.sort(key=lambda x: x[1])

            # Seleccionar solo la mitad de los agentes más cercanos a un POI
            num_agents_to_assign = 4
            agents_to_assign = agent_distances[:num_agents_to_assign]

            # Asignar los agentes seleccionados a los POIs
            for agent, _, closest_portrait in agents_to_assign:
                portrait_assignments[closest_portrait].append(agent)

            # Actualizar el estado de los agentes asignados a los POIs
            for portrait, agents in portrait_assignments.items():
                for agent in agents:

                    # Si el agente está asignado a un POI, debe buscar el POI
                    agent.state = 'heading_to_portrait'

            # Los agentes que no están asignados a un POI deben apagar el fuego
            for agent in self.schedule.agents:
                if agent.state != 'heading_to_portrait' and agent.state != 'heading to entrance':
                    agent.state = 'capturing'

    def place_new_portrait(self, index_of_new_portrait):

        prob_of_victim = 0.5

        # Seleccionar una ficha basada en las probabilidades actuales
        portrait_type = random.choices(['victim', 'alarm'], weights=[prob_of_victim, 1 - prob_of_victim])[0]
        x = self.random.randrange(self.grid.width)
        y = self.random.randrange(self.grid.height)

        # Asegurarse de que (x, y) no esté en una celda con fuego
        while self.ghosts_grid[x][y] != 0 and (x, y) not in self.entrances:
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)

        self.moves.append(["portraitC", x, y])

        # Convertir la tupla a una lista, modificar y convertir de nuevo a tupla
        current_portrait = list(self.portrait[index_of_new_portrait])
        current_portrait[0] = x
        current_portrait[1] = y
        current_portrait[2] = True if portrait_type == 'victim' else False
        current_portrait[3] = False
        self.portrait[index_of_new_portrait] = tuple(current_portrait)

        self.portrait_grid[x][y] = 3

        # Actualizar contadores de victimas o alarmas residuales
        if portrait_type == 'victim':
            self.residual_victims -= 1
        else:
            self.residual_alarms -= 1


    def is_open(self, y, x):
        y2, x2 = self.doors_connected[y, x]
        if y > y2:
            open = self.doors[y2, x2, y, x]
        elif y < y2:
            open = self.doors[y, x, y2, x2]
        if x > x2:
            open = self.doors[y2, x2, y, x]
        elif x < x2:
            open = self.doors[y, x, y2, x2]

        return open


    def spawn(self):
        x = self.random.randrange(self.grid.width)
        y = self.random.randrange(self.grid.height)
        haunting = False

        # Si cae en una celda vacía
        if self.ghosts_grid[x][y] == 0:
            neighbors = self.grid.get_neighborhood((x,y), moore=False, include_center=False)

            while neighbors and not haunting:
                pos = self.random.choice(neighbors)

                magX = pos[0] - x
                magY = pos[1] - y

                # Arriba
                direction = 0
                if magY > 0:
                    # Derecha
                    direction = 1
                elif magY < 0:
                    # Izquierda
                    direction = 3
                elif magX > 0:
                    # Abajo
                    direction = 2

                # Si es una puerta abierta, revisa
                if self.walls_grid[x, y][direction] == 'door' and self.is_open(x, y):
                    if self.ghosts_grid[pos[0]][pos[1]] == 2:
                        haunting = True

                # Si lo que hay es diferente a una entrada o una puerta
                elif self.walls_grid[x, y][direction] != 'entrance' and  self.walls_grid[x, y][direction] != 'door':

                    # Si no se encuentra con una pared, revisa
                    if self.walls_grid[x, y][direction] == 0:
                        if self.ghosts_grid[pos[0]][pos[1]] == 2:
                            haunting = True

                neighbors = [x for x in neighbors if x != pos]


            if haunting:

                cell_contents = self.grid.get_cell_list_contents((x, y))
                for obj in cell_contents:
                    if isinstance(obj, LuigiAgent):
                        new_pos = self.random.choice(self.entrances)

                        self.grid.move_agent(obj, new_pos)
                        self.moves.append(["teleport", new_pos[0], new_pos[1], obj.unique_id])

                        self.assign_initial_statement()

                # Si esa celda tenía un POI
                if self.portrait_grid[x][y] > 0:
                    self.moves.append(["portraitD", x, y])

                    for i in range(3):
                        if self.portrait[i][0] == x and self.portrait[i][1] == y:

                            # Si el POI era una victima
                            if self.portrait[i][2]:
                                self.total_deaths += 1
                                self.portrait_grid[x][y] = 0
                                self.place_new_portrait(i)
                            else:
                                self.portrait_grid[x][y] = 0
                                self.place_new_portrait(i)

                self.moves.append(["ghost", x, y])
                self.portrait_grid[x][y] = 2
            else:
                self.moves.append(["knocked_ghost", x, y])
                self.ghosts_grid[x][y] = 1

        #Si cae en una celda con humo
        elif self.ghosts_grid[x][y] == 1:

            entrances_wo_ghost = [pos for pos in self.entrances if self.ghosts_grid[pos[0]][pos[1]] != 2]

            # Obtener todos los agentes en la celda de la posición `pos`
            agents = self.grid.get_cell_list_contents([(x, y)])

            # Iterar sobre cada agente en la lista de agentes
            for agent in agents:

                # Seleccionar una nueva posición al azar de la lista de entradas
                if len(entrances_wo_ghost) > 0:
                    new_pos = self.random.choice(entrances_wo_ghost)
                else:
                    new_pos = self.random.choice(self.entrances)

                # Mover el agente a la nueva posición seleccionada
                self.grid.move_agent(agent, new_pos)
                self.moves.append(["teleport", new_pos[0], new_pos[1], agent.unique_id])

                self.assign_initial_statement()

            # Si esa celda tenía un POI
            if self.portrait_grid[x][y] > 0:
                self.moves.append(["portraitD", x, y])
                for i in range(3):
                    if self.portrait[i][0] == x and self.portrait[i][1] == y:

                        # Si el POI era una victima
                        if self.portrait[i][2]:
                            self.total_deaths += 1
                            self.portrait_grid[x][y] = 0
                            self.place_new_portrait(i)
                        else:
                            self.portrait_grid[x][y] = 0
                            self.place_new_portrait(i)
            self.moves.append(["ghost", x, y])
            self.ghosts_grid[x][y] = 2

        # Si cae en una celda con fuego
        elif self.ghosts_grid[x][y] == 2:
            neighbors = self.grid.get_neighborhood((x,y), moore=False, include_center=False)

            self.moves.append(["red_ghost", x, y])
            for neigh_x, neigh_y in neighbors:
                self.explosion((x,y), neigh_x - x, neigh_y - y)


    def explosion(self, pos, x, y):

        # Obtengo el índice en el arreglo ce la casilla que corresponde a la direccion adecuada
        # Arriba
        direction = 0
        if y > 0:
            # Derecha
            direction = 1
        elif y < 0:
            # Izquierda
            direction = 3
        elif x > 0:
            # Abajo
            direction = 2

        # Primero reviso si hay algún obstáculo en la dirección a la que me voy a mover
        # Si se encuentra con una puerta que está cerrada
        if self.walls_grid[pos[0], pos[1]][direction] == 'door' and not self.is_open(pos[0], pos[1]):
            self.destroyDoor(pos, direction)
            return

        # Si lo que hay es diferente a una entrada o una puerta
        elif self.walls_grid[pos[0], pos[1]][direction] != 'entrance' and  self.walls_grid[pos[0], pos[1]][direction] != 'door':
            # Si se encuentra con una pared
            if self.walls_grid[pos[0], pos[1]][direction] > 0:
                self.damageWall(pos, direction)
                return

        # Si no hay nada en el camino reviso la celda siguiente
        new_X = pos[0] + x
        new_Y = pos[1] + y
        if not self.grid.out_of_bounds((new_X, new_Y)):
            # Si se encuentra en un espacio vacío o con humo
            if self.ghosts_grid[new_X][new_Y] <= 1:

                entrances_wo_ghost = [pos for pos in self.entrances if self.ghosts_grid[pos[0]][pos[1]] != 2]
                # Obtener todos los agentes en la celda de la posición `pos`
                agents = self.grid.get_cell_list_contents([(new_X, new_Y)])

                # Iterar sobre cada agente en la lista de agentes
                for agent in agents:
                    # Seleccionar una nueva posición al azar de la lista de entradas
                    if len(entrances_wo_ghost) > 0:
                        new_pos = self.random.choice(entrances_wo_ghost)
                    else:
                        new_pos = self.random.choice(self.entrances)

                    # Mover el agente a la nueva posición seleccionada
                    self.grid.move_agent(agent, new_pos)
                    self.moves.append(["teleport", new_pos[0], new_pos[1], agent.unique_id])

                    # Imprimir un mensaje indicando que el agente ha sido movido

                    self.assign_initial_statement()

                # Si esa celda tenía un POI
                if self.portrait_grid[new_X][new_Y] > 0:
                    self.moves.append(["portraitD", new_X, new_Y])

                    for i in range(3):
                        if self.portrait[i][0] == new_X and self.portrait[i][1] == new_Y:
                            # Si el POI era una victima
                            if self.portrait[i][2]:
                                self.total_deaths += 1
                                self.portrait_grid[new_X][new_Y] = 0
                                self.place_new_portrait(i)
                            else:
                                self.portrait_grid[new_X][new_Y] = 0
                                self.place_new_portrait(i)
                self.moves.append(["ghost", new_X, new_Y])
                self.ghosts_grid[new_X][new_Y] = 2

            # Si se encuentra con una casilla con fuego
            elif self.ghosts_grid[new_X][new_Y] == 2:
                self.moves.append(["explosion", new_X, new_Y])
                self.explosion((new_X, new_Y), x, y)


    def haunt(self):
        knocked_ghosts = []
        for x in range(len(self.ghosts_grid)):
            for y in range(len(self.ghosts_grid[x])):
                if self.ghosts_grid[x][y] == 1:
                    neighbors = self.grid.get_neighborhood((x,y), moore=False, include_center=False)
                    haunting = False
                    while neighbors and not haunting:
                        pos = self.random.choice(neighbors)
                        magX = pos[0] - x
                        magY = pos[1] - y

                        # Arriba
                        direction = 0
                        if magY > 0:
                            # Derecha
                            direction = 1
                        elif magY < 0:
                            # Izquierda
                            direction = 3
                        elif magX > 0:
                            # Abajo
                            direction = 2

                        # Si es una puerta abierta, revisa
                        if self.walls_grid[x, y][direction] == 'door' and self.is_open(x, y):
                            if self.fires_grid[pos[0]][pos[1]] == 2:
                                knocked_ghosts.append((x,y))
                                haunting = True

                        # Si lo que hay es diferente a una entrada o una puerta
                        elif self.walls_grid[x, y][direction] != 'entrance' and  self.walls_grid[x, y][direction] != 'door':

                            # Si no se encuentra con una pared, revisa
                            if self.walls_grid[x, y][direction] == 0:
                                if self.ghosts_grid[pos[0]][pos[1]] == 2:
                                    knocked_ghosts.append((x,y))
                                    haunting = True

                        neighbors = [x for x in neighbors if x != pos]

        while knocked_ghosts:
            knocked_ghost = knocked_ghosts.pop()
            self.moves.append(["haunt", knocked_ghost[0], knocked_ghost[1]])
            self.ghosts_grid[knocked_ghost[0]][knocked_ghost[1]] = 2
            entrances_wo_ghost = [pos for pos in self.entrances if self.ghosts_grid[pos[0]][pos[1]] != 2]

            # Obtener todos los agentes en la celda de la posición `smoke`
            agents = self.grid.get_cell_list_contents([knocked_ghost])

            # Iterar sobre cada agente en la lista de agentes
            for agent in agents:

                # Seleccionar una nueva posición al azar de la lista de entradas
                if len(entrances_wo_ghost) > 0:
                    new_pos = self.random.choice(entrances_wo_ghost)
                else:
                    new_pos = self.random.choice(self.entrances)

                # Mover el agente a la nueva posición seleccionada
                self.grid.move_agent(agent, new_pos)
                self.moves.append(["teleport", new_pos[0], new_pos[1], agent.unique_id])

                # Imprimir un mensaje indicando que el agente ha sido movido
                self.assign_initial_statement()


            if self.portrait_grid[knocked_ghost[0]][knocked_ghost[1]] > 0:
                self.moves.append(["portraitD", knocked_ghost[0], knocked_ghost[1]])

                for i in range(3):
                    if self.portrait[i][0] == knocked_ghost[0] and self.portrait[i][1] == knocked_ghost[1]:

                        # Si el POI era una victima
                        if self.portrait[i][2]:
                            self.portrait_grid[knocked_ghost[0]][knocked_ghost[1]] = 0
                        else:
                            self.portrait_grid[knocked_ghost[0]][knocked_ghost[1]] = 0
                            self.place_new_portrait(i)

            neighbors = self.grid.get_neighborhood(knocked_ghost, moore=False, include_center=False)
            for neighbor in neighbors:
                if self.ghosts_grid[neighbor[0]][neighbor[1]] == 1:
                    knocked_ghosts.append(neighbor)


    def damageWall(self, pos, direction):
        if direction == 0:
            new_pos = (pos[0] - 1, pos[1])
            new_direction = 2
        elif direction == 1:
            new_pos = (pos[0], pos[1] + 1)
            new_direction = 3
        elif direction == 2:
            new_pos = (pos[0] + 1, pos[1])
            new_direction = 0
        elif direction == 3:
            new_pos = (pos[0], pos[1] - 1)
            new_direction = 1

        self.walls_grid[pos[0], pos[1]][direction] -= 1
        self.moves.append(["wall", pos[0], pos[1], direction, self.walls_grid[pos[0], pos[1]][direction]])

        if not self.grid.out_of_bounds(new_pos):
            self.walls_grid[new_pos[0], new_pos[1]][new_direction] -= 1
            self.moves.append(["wall", new_pos[0], new_pos[1], new_direction, self.walls_grid[new_pos[0], new_pos[1]][new_direction]])
        self.damage += 1


    def destroyDoor(self, pos, direction):
        if direction == 0:
            new_pos = (pos[0] - 1, pos[1])
            new_direction = 2
        elif direction == 1:
            new_pos = (pos[0], pos[1] + 1)
            new_direction = 3
        elif direction == 2:
            new_pos = (pos[0] + 1, pos[1])
            new_direction = 0
        elif direction == 3:
            new_pos = (pos[0], pos[1] - 1)
            new_direction = 1
        self.walls_grid[pos[0], pos[1]][direction] = 0
        self.moves.append(["door", pos[0], pos[1], direction])
        if not self.grid.out_of_bounds(new_pos):
            self.walls_grid[new_pos[0], new_pos[1]][new_direction] = 0
            self.moves.append(["door", new_pos[0], new_pos[1], new_direction])
        self.damage += 1


    def POI_pos(self):
        positions = []
        for poi in self.poi:
            positions.append([poi[0], poi[1]])
        return positions